In [1]:
#-------------------------------------IMPORTS-------------------------------------#
import torch
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA

In [2]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for word in title_word_list:
        word_features = extract_features(word)
        feature_matrix.append(word_features)
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0.] * NUM_FEATURES_PER_WORD] * (MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
# def cosine_similarity(question_one_batch_by_features, question_two_batch_by_features):
#     cosine_similarities = []
#     for idx in range(BATCH_SIZE):
#         vec_one = question_one_batch_by_features[idx]
#         vec_two = question_two_batch_by_features[idx]
#         numerator = np.dot(vec_one, vec_two)
#         denominator = LA.norm(vec_one)*LA.norm(vec_two)
#         value = numerator/denominator
#         cosine_similarities.append(value)
#     return cosine_similarities

# def cosine_similarity_worse(question_one_batch_by_features, question_two_batch_by_features):
#     print cosine_

In [3]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()


#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [20]:
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 1000 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES

KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 100
LEARNING_RATE = 1e-3
NUM_EPOCHS = 25
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200
BATCH_SIZE = 50
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)

In [5]:
import time
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:NUM_TRAINING_EXAMPLES])):   # we should include all but this is just for simplicity 
    if i%1000 == 0:
        print i

    sample = train_titles_only[i]
    
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
    
#     print "target_title: {}".format(target_title)
#     print "positive title: {}".format(positive_title)
#     print "negative titles: {}".format(negative_titles)
    
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
#     print "Target features shape: {}".format(target_features.shape)
#     print "Positive features shape: {}".format(positive_features.shape)
#     print "Negative features[0] shape: {}".format(n_negative_features[0].shape)
#     print "Num negative features: {}".format(len(n_negative_features)) 
    
    all_features = [target_features, positive_features] + n_negative_features
    features.append(all_features)

start = time.time()
print "Done Loop"
features = np.array(features) #speed up the following operations
targets = torch.LongTensor(len(features), 1).zero_()
training_dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
train_loader = data_utils.DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
print "Succesfully made the data batcher"

0
Done Loop
Succesfully made the data batcher


In [7]:
#-------------------------------------CNN-------------------------------------#
class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
#             nn.ReLU()
            nn.Tanh()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [19]:
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

net = CNN(INPUT_SIZE, HIDDEN_SIZE)
criterion = nn.MultiMarginLoss(p=1, margin=0.5, weight = None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

cos_torch = nn.CosineSimilarity(dim=0, eps=1e-6)

print "Beginning training..."
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        batch_cos_similarities = []
        batch_num = idx
        
        target_question_features = sample[:, 0]
        positive_question_features = sample[:, 1]
        N_negative_question_features = sample[:, 2:]
        
        target_question_net_output = net(Variable(target_question_features))
        positive_question_net_output = net(Variable(positive_question_features))
        N_negative_question_net_outputs = [net(Variable(negative)) for negative in N_negative_question_features]
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative) for negative in N_negative_question_features]
                
        batch_cos_sims = []
        #50x600x38 --> 50x600x1. Averaging across the net outputs to get a single vector for each question.
        for sample_num in range(BATCH_SIZE):
            target_question_length = target_question_lengths[sample_num]
            positive_question_length = positive_question_lengths[sample_num]
            N_negative_question_lengths = N_negative_questions_lengths[sample_num]
            N_negative_question_outputs = N_negative_question_net_outputs[sample_num]
            
            target_question_vector = target_question_net_output.data.numpy()[sample_num,:,:target_question_length].mean(axis = 1)
            positive_question_vector = positive_question_net_output.data.numpy()[sample_num,:,:positive_question_length].mean(axis = 1)
            
            N_negative_question_vectors = [negative.data.numpy()[:,:N_negative_question_lengths[idx]].mean(axis = 1) for idx, negative in enumerate(N_negative_question_outputs)]
        
            cosine_similarity_pos = np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), positive_question_vector.reshape(1, -1)))
            cosine_similarity_neg = [np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), negative.reshape(1, -1))) for negative in N_negative_question_vectors]
            
            cosine_sims = np.array([cosine_similarity_pos] + cosine_similarity_neg).flatten()
#             print cosine_sims
            batch_cos_sims.append(cosine_sims)
            if sample_num == 5:
                print "Positive question vector: {}".format(positive_question_vector)
                print "Negative question vector[0]: {}".format(N_negative_question_vectors[0])
                print "Cosine similarities for a random sample in the batch: \n{}".format(cosine_sims)
                
#                 targ_ques_vec_Var = Variable(torch.FloatTensor(target_question_vector))
#                 pos_ques_vec_Var = Variable(torch.FloatTensor(positive_question_vector))
#                 N_neg_ques_vec_Vars = [Variable(torch.FloatTensor(negative)) for negative in N_negative_question_vectors]
                
#                 pos_torch_cos_sim = cos_torch(targ_ques_vec_Var, pos_ques_vec_Var).data.numpy()[0]
#                 neg_torch_cos_sims = [cos_torch(targ_ques_vec_Var, neg_ques_vec_Var).data.numpy()[0]
#                                       for neg_ques_vec_Var in N_neg_ques_vec_Vars]
#                 all_torch_sims = [pos_torch_cos_sim] + neg_torch_cos_sims
                
#                 print "Cosine similarities with torch function: \n{}".format(all_torch_sims)
        
        optimizer.zero_grad()
        
        batch_labels = Variable(torch.from_numpy(label.numpy().squeeze()), requires_grad = True)
        loss = criterion(Variable(torch.FloatTensor(np.array(batch_cos_sims))), batch_labels)
            
        loss.backward()
            
        running_loss += loss.data[0]
            
        optimizer.step()
        print "Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.data[0])

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
            


Beginning training...
Positive question vector: [  3.96156795e-02  -3.05480864e-02  -8.79894383e-03  -9.02584381e-03
  -1.59060992e-02  -1.66125353e-02   2.01290403e-03   5.09378836e-02
   8.34244862e-03  -1.87399536e-02   4.77511510e-02  -2.77796481e-02
   2.44743228e-02   2.59938445e-02   1.00994017e-02   4.71583270e-02
   6.13757744e-02   3.36974845e-05   3.21366750e-02  -3.76628414e-02
   7.77998287e-03  -1.00696934e-02   2.49127559e-02  -5.89459762e-02
   2.67751068e-02   1.14410250e-02  -2.15011127e-02  -6.78586140e-02
   7.57582253e-03   3.92689668e-02   4.31511737e-02   2.19146777e-02
   3.27810571e-02  -1.92014687e-02   1.69187766e-02   2.77004242e-02
   3.64312902e-02   3.46413590e-02   4.29477394e-02  -2.54286621e-02
  -4.64846343e-02   9.60227940e-03   1.99772343e-02   1.25879617e-02
   6.27870038e-02  -1.98157281e-02  -4.83097602e-03   4.09801342e-02
   3.20103876e-02   2.27600839e-02  -7.90984742e-03   8.40374175e-03
  -2.86008473e-02   2.58059353e-02  -4.23787646e-02  -1

Positive question vector: [  4.74374555e-02  -1.23420348e-02  -4.35530879e-02  -6.78682933e-03
   1.94990542e-02  -4.27977741e-02   1.71574857e-02   1.15170572e-02
  -2.73437658e-03   8.30811169e-03   4.02104780e-02  -1.94423960e-03
  -1.30905584e-03  -8.80356412e-03   2.21476462e-02   2.13947352e-02
   1.07101561e-03  -6.57471502e-03  -2.32965089e-02  -7.05689266e-02
   1.03663618e-03  -1.97478980e-02   2.72998605e-02  -4.02434766e-02
   1.75205781e-03   1.48749370e-02  -2.67887153e-02  -1.22557897e-02
  -1.65951364e-02   6.71945959e-02   4.89288270e-02  -2.22129021e-02
   6.82446510e-02  -2.97508240e-02   8.26655701e-02   4.52205725e-02
   2.49323212e-02   6.34347051e-02   1.24209123e-02  -1.36990016e-02
  -2.75624543e-02  -4.00070986e-03   6.80055395e-02   4.50199330e-03
   1.59017462e-02   7.91063160e-03  -1.47553151e-02   4.25692759e-02
   2.88029425e-02   6.41086278e-03   2.09443066e-02   6.00055121e-02
  -1.59426536e-02   2.43594684e-02  -5.85013106e-02  -8.10889251e-05
  -1.662

Positive question vector: [ 0.05046416 -0.01056958 -0.01226772 -0.0184491  -0.01054639 -0.03273011
  0.01233675  0.028692    0.00013838 -0.00887234  0.03165752 -0.01722682
 -0.01105789  0.00511538  0.03096195  0.04522612  0.05124484 -0.00240863
  0.00099267 -0.02450622  0.01576219 -0.02076115  0.02107451 -0.04721873
  0.03225632 -0.00468883 -0.02164305 -0.03439265  0.00581963  0.04848598
  0.03831744 -0.00716874  0.04165821 -0.03015737  0.03684478  0.00574234
  0.01258845  0.05369417  0.04404403 -0.0008678  -0.02385022  0.02514346
  0.0254278   0.013717    0.0196282  -0.01834681 -0.00999746  0.0039671
  0.02223533 -0.0168212  -0.00463527  0.03520678 -0.01497866  0.03692455
 -0.04129039 -0.01591642 -0.0315455   0.0275054   0.03162793  0.00604742
 -0.01480263  0.03327965  0.02082052  0.00081799 -0.02361792 -0.0083603
  0.01479512 -0.0022999   0.01660602  0.00157356  0.01602754  0.01100561
  0.01035263 -0.06096727  0.009633    0.02773308 -0.00743749  0.02650071
  0.033026   -0.01588249  0

Positive question vector: [ 0.04470041 -0.02791521 -0.02454092 -0.04215149 -0.0002658  -0.03958175
  0.00892876  0.01768217  0.00598098 -0.02276506  0.01964127 -0.01747988
  0.00487891  0.00262297  0.05054619  0.02554743  0.03089802  0.01028755
  0.00379127 -0.04390848  0.0152118  -0.03449258  0.01603142 -0.02530687
  0.0066265   0.03340056 -0.03262218 -0.02330771 -0.00256629  0.037535
  0.01997897  0.0368217   0.0473096  -0.04288883  0.05463724  0.03210187
  0.01897343  0.05714624  0.01947918 -0.025633   -0.04287287  0.00630547
  0.05754545  0.02317115  0.04715874 -0.01010542 -0.01588122  0.02382669
  0.03039611 -0.00306294 -0.00089847  0.05217132 -0.01490773  0.01129274
 -0.02323816 -0.00902748 -0.00710118 -0.00859543  0.0202414  -0.0123329
  0.00603719  0.01648862  0.03623216  0.02841326 -0.05242421 -0.01814681
  0.01174037  0.0187705   0.01473388 -0.00358475  0.02020722  0.01250099
  0.01389685 -0.05728882  0.00014773 -0.0042403   0.00568318  0.03217113
  0.02483062 -0.01924232  0.

Epoch: 0, Batch: 11, Loss: 0.428935557604
Positive question vector: [  6.59867525e-02  -3.31142806e-02  -1.38521930e-02   1.70957576e-02
  -3.15441797e-03  -3.49035710e-02   2.28131395e-02   1.48922345e-02
   8.12478270e-03   5.59795462e-03   5.65093532e-02  -1.74448621e-02
   4.04452952e-03   1.21640749e-02   2.85175033e-02   5.00543341e-02
   3.54435965e-02  -9.22960136e-03  -6.44275395e-04  -4.48887423e-02
   2.25714091e-02  -2.98074540e-02   2.92206127e-02  -3.51865888e-02
   3.29038240e-02   2.56518833e-03  -2.35737376e-02  -2.64166668e-02
   3.15678492e-03   4.77789640e-02   4.07413989e-02   2.18760385e-03
   4.07832600e-02  -3.25222462e-02   6.01871014e-02  -9.44501534e-03
   7.25689251e-03   4.06682789e-02   2.78982241e-02   1.69322491e-02
  -2.07735188e-02   1.26788160e-02   2.68500820e-02   2.42751581e-03
   1.76198203e-02  -2.25791801e-02   3.70254903e-03   5.12011796e-02
   4.90367077e-02   2.12944392e-02  -1.02086030e-02   3.78760695e-03
  -1.38636921e-02   4.19586934e-02 

...

Epoch: 6, Batch: 17, Loss: 0.421286314726
Positive question vector: [ 0.03498089 -0.00150414 -0.05420489  0.01789093 -0.01214057 -0.03815471
 -0.00376644  0.02090265  0.01095352 -0.00767555  0.04777096  0.01542128
  0.02383734 -0.00975463  0.03167684  0.04403521  0.03260131  0.01206509
 -0.02372491 -0.02516735  0.00808746 -0.018552    0.00395877 -0.03784921
  0.03764314  0.02482692 -0.02754165 -0.04990389  0.02120525  0.03975525
  0.04301573 -0.01502485  0.02505667 -0.0438318   0.03450565  0.0381273
  0.01170475  0.03112482 -0.01851958 -0.00318991 -0.01317841  0.00416771
  0.05082203  0.00213362  0.0448401  -0.0483878  -0.01054589  0.02669768
  0.02402293  0.01090333  0.00192136  0.02894065 -0.03391563  0.02865179
 -0.03710251  0.01970374 -0.03878693  0.01523461  0.04641271 -0.00127306
  0.02238348  0.04787121  0.03577897  0.00514631 -0.03491578 -0.0320207
  0.02390923  0.03711546  0.02954159  0.01272839  0.03799099  0.00300147
 -0.00084846 -0.04815863 -0.01462019  0.01163378 -0.024638

Positive question vector: [ 0.02181051 -0.03628014 -0.00113326 -0.00495973  0.03262356 -0.02915082
  0.02034412  0.06514678  0.01255189 -0.03104017  0.03758005 -0.0051237
  0.02212996  0.00806666  0.03216116  0.02668235  0.0526184  -0.02709221
  0.03531339 -0.01194702  0.02617429 -0.03549615  0.02570435 -0.03682376
  0.01643337 -0.03357074 -0.0082005  -0.06844404  0.00736412  0.04566205
  0.05558248  0.0143396   0.01858254 -0.02394875  0.09183367  0.02159259
  0.02328019  0.05568258  0.04688113  0.01859313 -0.03751291  0.0004156
  0.01337024 -0.01802001  0.00613495 -0.04869089  0.00822264  0.02523398
  0.03549447  0.02761256 -0.00161903  0.02816529 -0.00105459  0.03698302
 -0.06215553  0.0006471  -0.01840416  0.02314852  0.04719562  0.00381923
 -0.00306119  0.0228537   0.02001671 -0.00693949 -0.04253717 -0.03462251
 -0.00438274  0.00950019  0.01721652 -0.00924106  0.0339823   0.00668529
  0.01865507 -0.0286163   0.01847995  0.02729769 -0.0184036   0.0192472
  0.05901712 -0.0215333   0.

Positive question vector: [ 0.02738751 -0.02230636 -0.04714484 -0.00118562 -0.03177593 -0.00174127
 -0.00083194  0.03788076  0.03818996  0.00675812  0.02782901 -0.01790781
  0.03591348 -0.03882803  0.02204245  0.03993576  0.04894653 -0.03442884
  0.03144924 -0.03199665  0.01911025 -0.04475524  0.02293324 -0.03458701
  0.03021071  0.05073353 -0.04461045 -0.05117403 -0.00885864  0.06347917
  0.01013681  0.00845128  0.05669093 -0.05990512  0.05153924  0.01987926
  0.03426267  0.04375293  0.01408755 -0.0080036  -0.01750944  0.0002394
  0.0501853  -0.01626316  0.00746053 -0.02405312 -0.01536048  0.0020101
  0.01719799  0.00262501 -0.00750449  0.07463452 -0.00326647 -0.0052973
 -0.04081377 -0.0007577  -0.03582898 -0.00133784  0.03922083 -0.03848793
 -0.00376664  0.03050449  0.02800496  0.02713341 -0.05369685 -0.03134738
 -0.00028547  0.01281726 -0.00077015  0.0274233   0.01473008  0.01508706
  0.03349449 -0.05511111  0.02638314  0.00355764 -0.00653966  0.03031187
  0.02786675 -0.0171149   0.

Epoch: 7, Batch: 6, Loss: 0.42311874032
Positive question vector: [ 0.03965024 -0.04602839 -0.01122592 -0.02425629 -0.01376082 -0.03446124
  0.01390927  0.01994448  0.00446549 -0.00860989  0.06505485 -0.0253666
  0.02494097 -0.01522778  0.02374397  0.05014213  0.05209624 -0.02651751
  0.01999586 -0.02358231  0.02283119 -0.00919649 -0.00951996 -0.01831193
  0.04027363 -0.00550526 -0.04608436 -0.0256375   0.01155138  0.03185187
  0.03517723  0.01607574  0.02201973 -0.04870367  0.04274552  0.00220963
  0.03133722  0.02180997  0.02077797 -0.00908621 -0.03724359  0.01902126
  0.01277214 -0.02160053  0.0303216  -0.02814419 -0.02365093  0.03607846
  0.03710446 -0.01528976 -0.03546848  0.00279087  0.00301349  0.03145532
 -0.05174651  0.03692662 -0.06012611 -0.00328133  0.033303   -0.02917515
 -0.00543646  0.02412155  0.01699099 -0.01234068 -0.04069063 -0.02659785
 -0.00184063  0.00827708  0.00781874 -0.00773873  0.0317854   0.02538234
  0.02080817 -0.03837602  0.01552385  0.00806057 -0.0077821

KeyboardInterrupt: 